In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [2]:
unique = Train['Country Name'].unique()

In [3]:
j= pd.read_csv('/kaggle/input/missing-prediction/data.csv')

In [4]:
missing_value2 = j.values

In [5]:
training_list=[]
training_list = [j for j in unique if j not in missing_value2]

In [6]:
unique_values = set(training_list)

In [7]:
Train=Train.replace('..', .01020)
Test=Test.replace('..', .01020) 

In [8]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

k=0
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,9,10]].astype(float)
    xt=Xtrain.values
    Ytrain=transposed_df[11].astype(float) 
    yt=Ytrain.values
    #logistic_regression = XGBRegressor()
  

   

       # Compile model
    
  
      # Reshape training data to fit the model input shape
   

      # Train the model
    if k == 0:
        k+=1
        model = Sequential([
              LSTM(64, return_sequences=True, input_shape=(Xtrain.shape[1], 1)),
              Dropout(0.2),  # Adding dropout for regularization
              LSTM(64, return_sequences=True),
              Dropout(0.2),
              LSTM(64),  # Last LSTM layer without return sequences
              Dropout(0.2),
              Dense(32, activation='relu'),  # Adding a dense layer for additional processing
              Dense(1)  # Output layer
              ])
        model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        Xtrain = xt.reshape((xt.shape[0], xt.shape[1], 1))
        model.fit( Xtrain,  yt, epochs=30, batch_size=32)
        model.save("/kaggle/working/my_lstm_model.h5")
    else:
        loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
        loaded_model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        loaded_model.fit(Xtrain, yt, epochs=30, batch_size=32)
        loaded_model.save("/kaggle/working/my_lstm_model.h5")
        
    # Generate some test sequences for predictio  
    
print(j)
        

2024-03-17 17:46:25.239029: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 17:46:25.239182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 17:46:25.450815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.0147
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0159
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0129
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0134
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0129
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0161
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0111
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0138
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0123
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0110
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0092
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0129
Epoch 13/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0133
Epoch 14/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0110
Epoch 15/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0098
Epoch 16/30
1/1 ━━━━━━━━━━━━━━━━━━━━